# AI Demand Prediction Demo

This notebook demonstrates the AI demand prediction functionality for the Smart CNG Pump Appointment System.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Import our demand predictor
import sys
sys.path.append('.')
from ai_models.demand_predictor import DemandPredictor

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Generate Sample Data

Let's create some sample historical data to train our model.

In [ ]:
# Generate sample historical data
np.random.seed(42)  # For reproducible results

# Create sample data for 3 pumps over 30 days
sample_data = []
pump_ids = ['PUMP001', 'PUMP002', 'PUMP003']
weathers = ['clear', 'cloudy', 'rainy']
traffics = ['low', 'medium', 'high']

start_date = datetime(2023, 1, 1)

for day in range(30):
    current_date = start_date + timedelta(days=day)
    
    # More bookings on weekdays
    is_weekday = current_date.weekday() < 5
    base_bookings = 15 if is_weekday else 8
    
    for pump_id in pump_ids:
        # Generate bookings for each hour (6 AM to 6 PM)
        for hour in range(6, 18):
            # Higher demand during rush hours
            if 7 <= hour <= 9 or 17 <= hour <= 19:
                hourly_multiplier = 1.5
            elif 12 <= hour <= 14:
                hourly_multiplier = 1.2
            else:
                hourly_multiplier = 1.0
                
            # Random variations
            demand = int(np.random.normal(base_bookings * hourly_multiplier, 3))
            demand = max(0, demand)  # Ensure non-negative
            
            # Random weather and traffic conditions
            weather = np.random.choice(weathers)
            traffic = np.random.choice(traffics)
            
            sample_data.append({
                'pump_id': pump_id,
                'slot_date': current_date.date(),
                'slot_time': f'{hour:02d}:00:00',
                'demand_count': demand,
                'weather': weather,
                'traffic': traffic
            })

# Convert to DataFrame
df = pd.DataFrame(sample_data)
print(f"Generated {len(df)} sample records")
df.head(10)

## Visualize Sample Data

Let's visualize some patterns in our sample data.

In [ ]:
# Plot demand by hour of day
hourly_demand = df.groupby(df['slot_time'].str[:2])['demand_count'].mean()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
hourly_demand.plot(kind='bar')
plt.title('Average Demand by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Average Demand')
plt.xticks(rotation=0)

# Plot demand by day of week
df['day_of_week'] = pd.to_datetime(df['slot_date']).dt.day_name()
weekday_demand = df.groupby('day_of_week')['demand_count'].mean()

plt.subplot(1, 2, 2)
weekday_demand.plot(kind='bar')
plt.title('Average Demand by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Average Demand')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Train the Demand Prediction Model

Now let's train our demand prediction model with the sample data.

In [ ]:
# Initialize the demand predictor
predictor = DemandPredictor()

# Train the model
print("Training the demand prediction model...")
metrics = predictor.train(df)

print(f"Model trained successfully!")
print(f"Training samples: {metrics['samples']}")
print(f"RMSE: {metrics['rmse']:.2f}")

## Make Predictions

Let's use our trained model to make some predictions.

In [ ]:
# Make predictions for a specific pump and time
test_date = datetime(2023, 2, 1)  # A future date
test_pump = 'PUMP001'

print(f"Demand predictions for {test_pump} on {test_date.date()}")
print("-" * 50)

# Predict demand for each hour
predictions = []
for hour in range(6, 18):
    slot_time = f"{hour:02d}:00"
    
    # Predict with different conditions
    pred_clear_low = predictor.predict_demand(test_pump, test_date, slot_time, 'clear', 'low')
    pred_rainy_high = predictor.predict_demand(test_pump, test_date, slot_time, 'rainy', 'high')
    
    predictions.append({
        'hour': hour,
        'clear_low': pred_clear_low,
        'rainy_high': pred_rainy_high
    })
    
    print(f"{slot_time}: Clear/Low={pred_clear_low:.1f}, Rainy/High={pred_rainy_high:.1f}")

## Visualize Predictions

Let's visualize our predictions to see the patterns.

In [ ]:
# Convert predictions to DataFrame for plotting
pred_df = pd.DataFrame(predictions)

plt.figure(figsize=(12, 6))
plt.plot(pred_df['hour'], pred_df['clear_low'], marker='o', label='Clear Weather, Low Traffic')
plt.plot(pred_df['hour'], pred_df['rainy_high'], marker='s', label='Rainy Weather, High Traffic')
plt.xlabel('Hour of Day')
plt.ylabel('Predicted Demand')
plt.title(f'Demand Predictions for {test_pump}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(6, 18))
plt.show()

## Find Optimal Slots

Let's identify the optimal time slots (lowest predicted demand) for a given day.

In [ ]:
# Find optimal slots (lowest demand)
optimal_slots = pred_df.nsmallest(5, 'clear_low')

print(f"Top 5 optimal time slots for {test_pump} (Clear Weather, Low Traffic):")
print("-" * 60)
for _, row in optimal_slots.iterrows():
    hour = int(row['hour'])
    demand = row['clear_low']
    print(f"  {hour:02d}:00 - Predicted demand: {demand:.1f}")

## Summary

Our AI demand prediction model can:

1. Learn patterns from historical booking data
2. Predict demand based on time, weather, and traffic conditions
3. Help identify optimal time slots for customers
4. Assist in capacity planning for pump stations

In a production environment, this model would be retrained regularly with new data to improve accuracy.